In [1]:
### Assignment 1 in DAT510. Cryptanalysis of primitive ciphers ###
### Part 2. Simplified DES

#Import all constants

key_length = 10
subkey_length = 8

P10 = (3,5,2,7,4,10,1,9,8,6)
P8 = (6,3,7,4,8,5,10,9)
IP = (2,6,3,1,4,8,5,7)
Ipinverse = (4,1,3,5,7,2,8,6)
EP = (4,1,2,3,2,3,4,1)
S0 = (1,0,3,2,3,2,1,0,0,2,1,3,3,1,3,2)
S1 = (0,1,2,3,2,0,1,3,3,0,1,0,2,1,0,3)
P4 = (2,4,3,1)

In [1]:
def Ipinverse_func(inputByte):
    return perm(inputByte, Ipinverse)

In [3]:
def ip(inputByte):
    return perm(inputByte, IP)

In [4]:
def key_gen(key):
    keyList = [(key & 1 << i) >> i for i in reversed(range(key_length))]
    permKeyList = [None] * key_length
    for index, elem in enumerate(P10):
        permKeyList[index] = keyList[elem - 1]
    shiftedOnceKey = leftShift(permKeyList)
    shiftedTwiceKey = leftShift(leftShift(shiftedOnceKey))
    subKey1 = subKey2 = 0
    for index, elem in enumerate(P8):
        subKey1 += (128 >> index) * shiftedOnceKey[elem - 1]
        subKey2 += (128 >> index) * shiftedTwiceKey[elem - 1]
    return (subKey1, subKey2)

In [5]:
def decrypt(key, ciphertext):
    data = fk(key_gen(key)[1], ip(ciphertext))
    return Ipinverse_func(fk(key_gen(key)[0], swapNibbles(data)))

In [6]:
def encrypt(key, plaintext):
    data = fk(key_gen(key)[0], ip(plaintext))
    return Ipinverse_func(fk(key_gen(key)[1], swapNibbles(data)))

In [7]:
def fk(subKey, inputData):
    def F(sKey, rightNibble):
        aux = sKey ^ perm(swapNibbles(rightNibble), EP)
        index1 = ((aux & 0x80) >> 4) + ((aux & 0x40) >> 5) + \
                 ((aux & 0x20) >> 5) + ((aux & 0x10) >> 2)
        index2 = ((aux & 0x08) >> 0) + ((aux & 0x04) >> 1) + \
                 ((aux & 0x02) >> 1) + ((aux & 0x01) << 2)
        sboxOutputs = swapNibbles((S0[index1] << 2) + S1[index2])
        return perm(sboxOutputs, P4)
 
    leftNibble, rightNibble = inputData & 0xf0, inputData & 0x0f
    return (leftNibble ^ F(subKey, rightNibble)) | rightNibble

In [8]:
def perm(inputByte, permTable):
    outputByte = 0
    for index, elem in enumerate(permTable):
        if index >= elem:
            outputByte |= (inputByte & (128 >> (elem - 1))) >> (index - (elem - 1))
        else:
            outputByte |= (inputByte & (128 >> (elem - 1))) << ((elem - 1) - index)
    return outputByte

In [9]:
def swapNibbles(inputByte):
    return (inputByte << 4 | inputByte >> 4) & 0xff

In [10]:
def leftShift(keyBitList):
    shiftedKey = [None] * key_length
    shiftedKey[0:9] = keyBitList[1:10]
    shiftedKey[4] = keyBitList[0]
    shiftedKey[9] = keyBitList[5]
    return shiftedKey

In [11]:
if __name__ == '__main__':
    choice = ''
    while choice != "0":
        choice = input('Enter e/E for Encrypt, d/D for Decrypt and 0 to exit: ')
        if choice.lower()=='e':
            k = int(input("Input binary value of Raw Key: "), 2)
            pt = int(input("Input binary value of Plaintext: "), 2)
            print("encrypted result: ",bin(encrypt(k, pt)))
        elif choice.lower()=='d':
        #elif choice.lower().startswith('d'):
            k1 = int(input("Input binary value of Raw Key: "), 2)
            ct = int(input("Input binary value of Ciphertext: "), 2)
            print("decrypted result: ", bin(decrypt(k1, ct)))
        elif choice != '0':
            print ('You have entered an invalid input, please try again. \n\n')
    exit()

Enter e/E for Encrypt, d/D for Decrypt and 0 to exit: e
Input binary value of Raw Key: 1111111111
Input binary value of Plaintext: 00000000
encrypted result:  0b11101011
Enter e/E for Encrypt, d/D for Decrypt and 0 to exit: 0
